#Assignment #1
##Machine Learning - Regression
###Rhichard Koh

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

california_housing = fetch_california_housing(as_frame=True)

In [ ]:
X = california_housing.data[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms']]
y = california_housing.target

In [ ]:
X = X[:1000]
y = y[:1000]

In [ ]:
from sklearn.model_selection import train_test_split

SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20,random_state=SEED)

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train,y_train)

y_pred = reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

result = pd.DataFrame({'Prediction':y_pred,'Actual':y_test})
print(f'MAE: {mean_absolute_error(y_test,y_pred)}')
print(f'MSE: {mean_squared_error(y_test,y_pred)}')
print(result)
result.to_excel('T Statistic.xlsx', sheet_name = 'New_sheet')

MAE: 0.38687590625585877
MSE: 0.23823690215175106
     Prediction  Actual
521    2.424104   3.353
737    1.784124   1.816
740    2.103969   1.840
660    1.913637   1.947
411    2.859023   3.271
..          ...     ...
408    3.451053   3.629
332    1.553184   0.972
208    1.198222   1.227
613    2.406288   2.519
78     0.801135   1.125

[200 rows x 2 columns]


In [ ]:
import numpy as np
from scipy import stats

result['y bar'] = result['Actual'].mean()
result['SSE'] = ((result['Prediction'] - result['Actual'])**2).sum()
result['SSE - y bar'] = result['SSE'] - result['y bar']
result['SSE - y hat'] = result['SSE'] - result['Prediction']
t_statistic, p_value = stats.ttest_ind(result['SSE - y bar'], result['SSE - y hat'], alternative='two-sided')

print(result)

print(f't_statistic: {t_statistic}')
print(f'p_value: {p_value}')

threshold = 0.05

if p_value > threshold:
  print(f'P value is greater than {threshold} therefore we reject the alternative hyptothesis and accept the null hypothesis.')
else:
  print(f'P value is less than {threshold} therefore we accept the alternative hyptothesis and reject the null hypothesis.')

     Prediction  Actual   y bar       SSE  SSE - y bar  SSE - y hat
521    2.424104   3.353  1.9572  47.64738     45.69018    45.223277
737    1.784124   1.816  1.9572  47.64738     45.69018    45.863256
740    2.103969   1.840  1.9572  47.64738     45.69018    45.543412
660    1.913637   1.947  1.9572  47.64738     45.69018    45.733744
411    2.859023   3.271  1.9572  47.64738     45.69018    44.788357
..          ...     ...     ...       ...          ...          ...
408    3.451053   3.629  1.9572  47.64738     45.69018    44.196327
332    1.553184   0.972  1.9572  47.64738     45.69018    46.094196
208    1.198222   1.227  1.9572  47.64738     45.69018    46.449159
613    2.406288   2.519  1.9572  47.64738     45.69018    45.241092
78     0.801135   1.125  1.9572  47.64738     45.69018    46.846246

[200 rows x 6 columns]
t_statistic: 2.826689779073848
p_value: 0.00494044098572805
P value is less than 0.05 therefore we accept the alternative hyptothesis and reject the null hypoth

this means y bar and y hat are statistically different

In [ ]:
coef = reg.coef_
intercept = reg.intercept_
print(coef)
print(intercept)

[ 0.41960584  0.00547113 -0.06695713  0.45090694]
0.1791415070272082


In [ ]:
ANOVA = X_test * coef

In [ ]:
ANOVA

,MedInc,HouseAge,AveRooms,AveBedrms
521,1.980036,0.284499,-0.473289,0.453716
737,1.331577,0.207903,-0.405776,0.471279
740,1.622868,0.218845,-0.369215,0.452329
660,1.491573,0.196961,-0.375576,0.421538
411,2.430021,0.284499,-0.488229,0.453591
...,...,...,...,...
408,3.010840,0.284499,-0.497457,0.474030
332,0.902153,0.257143,-0.226286,0.441033
208,0.679678,0.164134,-0.291742,0.467011
613,1.931949,0.279028,-0.446210,0.462380


In [ ]:
ANOVA_table = pd.DataFrame()
ANOVA_table['MedInc = 0'] = ANOVA['HouseAge'] + ANOVA['AveRooms'] + ANOVA['AveBedrms'] + intercept
ANOVA_table['HouseAge = 0'] = ANOVA['MedInc'] + ANOVA['AveRooms'] + ANOVA['AveBedrms'] + intercept
ANOVA_table['AveRooms = 0'] = ANOVA['MedInc'] + ANOVA['HouseAge'] + ANOVA['AveBedrms'] + intercept
ANOVA_table['AveBedrms = 0'] = ANOVA['MedInc'] + ANOVA['HouseAge'] + ANOVA['AveRooms'] + intercept
ANOVA_table.to_excel('ANOVA.xlsx', sheet_name = 'New_sheet')

In [ ]:
f_statistic, ANOVA_p_value = stats.f_oneway(ANOVA_table['MedInc = 0'], ANOVA_table['HouseAge = 0'], ANOVA_table['AveRooms = 0'], ANOVA_table['AveBedrms = 0'])

In [ ]:
print(f'f_statistic: {f_statistic}')
print(f'p_value: {ANOVA_p_value}')

if ANOVA_p_value > threshold:
  print(f'p_value is greater than {threshold} therefore we reject the alternative hyptothesis and accept the null hypothesis.')
else:
  print(f'p_value is less than {threshold} therefore we accept the alternative hyptothesis and reject the null hypothesis.')

f_statistic: 379.6965398705166
p_value: 4.948612982242282e-153
p_value is less than 0.05 therefore we accept the alternative hyptothesis and reject the null hypothesis.


this means that not every feature is statistically equal to one another.

In [ ]:
print(stats.ttest_ind(ANOVA_table['MedInc = 0'], ANOVA_table['HouseAge = 0'], alternative='two-sided'))
print(stats.ttest_ind(ANOVA_table['MedInc = 0'], ANOVA_table['AveRooms = 0'], alternative='two-sided'))
print(stats.ttest_ind(ANOVA_table['MedInc = 0'], ANOVA_table['AveBedrms = 0'], alternative='two-sided'))

print(stats.ttest_ind(ANOVA_table['HouseAge = 0'], ANOVA_table['AveRooms = 0'], alternative='two-sided'))
print(stats.ttest_ind(ANOVA_table['HouseAge = 0'], ANOVA_table['AveBedrms = 0'], alternative='two-sided'))

print(stats.ttest_ind(ANOVA_table['AveRooms = 0'], ANOVA_table['AveBedrms = 0'], alternative='two-sided'))

Ttest_indResult(statistic=-28.953565266184732, pvalue=5.358330767958238e-100)
Ttest_indResult(statistic=-38.26486440990877, pvalue=1.9730782351612314e-135)
Ttest_indResult(statistic=-23.293256755394207, pvalue=2.4646976979832037e-76)
Ttest_indResult(statistic=-8.036187444377521, pvalue=1.0648953659758067e-14)
Ttest_indResult(statistic=4.14090399261611, pvalue=4.226223800134049e-05)
Ttest_indResult(statistic=12.071207353603638, pvalue=8.370178297999249e-29)


none of the p values are greater than 0.05 therefore we reject the null hypothesis for every feature when one is set to 0. meaning none of them are statistically equal to one another.